In [1]:
using SummationByPartsOperators
using Plots
using Revise
using Distributions
using FFTW
using LinearAlgebra
using Base.Threads
using HDF5
using FileIO
using JLD2
 
includet("../PIC/PIC-1D/aux_functions/aux_functions.jl")
#includet("../PIC-1D/aux_functions/aux_functions.jl")
includet("local_aux_functions.jl")

In [ ]:

data_name = "data_3d_h_step_cut_small_F_full_T20.jld2"
@show data = load("Results/" * data_name)

In [4]:
m = 10 #data_steps - 1
par_grid = data["par_grid"]
#Box_x, J, n_fields = par_grid
par_evolv  = data["par_evolv"]
T, data_steps, int_steps, dt = par_evolv
run_name = data["run_name"]
size(data["u/u_00002"])
v = zeros(data_steps, size(data["u/u_00002"])...)
for m in 1:m
    tiempo = @sprintf("%05d", m)
    v[m,:,:,:,:] = data["u/u_$(tiempo)"]
end

In [ ]:
surface(v[10,1,:,:,150].-1 )
#heatmap(v[2,1,:,:,150].-1, aspectratio=1 )

In [ ]:

plot([get_norm_time(v,i)[1].-1 for i in 1:10], label="ϕ - 1" )
plot!([get_norm_time(v,i)[2] for i in 1:10], label="ϕ_t" )
plot!([get_norm_time(v,i)[3].-1 for i in 1:10], label="v - 1")
plot!([get_norm_time(v,i)[4] for i in 1:10], label="v_t")